In [50]:
import cv2
import os
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle

In [51]:
# Image Enhancing

def preprocess_img(img):
    resized_img = cv2.resize(img, (256,256))

    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    clahe = cv2.createCLAHE(3, (8,8))
    equalized_img = clahe.apply(gray)

    blurred_img = cv2.medianBlur(equalized_img, 5)

    hsv_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2HSV)

    hist_h = cv2.calcHist([hsv_img], [0], None, [180], [0, 180]) 
    hist_s = cv2.calcHist([hsv_img], [1], None, [256], [0, 256])
    hist_v = cv2.calcHist([hsv_img], [2], None, [256], [0, 256])

    hist_h = hist_h / np.linalg.norm(hist_h)
    hist_s = hist_s / np.linalg.norm(hist_s)
    hist_v = hist_v / np.linalg.norm(hist_v)

    color_hist = np.concatenate([hist_h, hist_s, hist_v]).flatten()

    return blurred_img, color_hist

In [52]:
def extract_glcm(img):
    resized_img = cv2.resize(img, (256,256))
    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    level = 8
    gray = np.floor(gray / (256 / level)).astype('uint8')

    glcm = graycomatrix(gray, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        levels=level, symmetric=True, normed=True)

    features = {
        'contrast': graycoprops(glcm, 'contrast').mean(),
        'dissimilarity': graycoprops(glcm, 'dissimilarity').mean(),
        'homogeneity': graycoprops(glcm, 'homogeneity').mean(),
        'energy': graycoprops(glcm, 'energy').mean(),
        'correlation': graycoprops(glcm, 'correlation').mean()
    }

    return features

In [53]:
sift = cv2.SIFT_create()

def extract_features(img):
    kp_sift, des_sift = sift.detectAndCompute(img, None)
    
    return kp_sift, des_sift

In [54]:
x = []
y = []
folders = os.listdir('Train/')

for idx, folder in enumerate(folders):
    folder_path = os.path.join('Train', folder)

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        
        if img is not None:
            final_img, color_hist = preprocess_img(img)
            kp_sift, des_sift = extract_features(final_img)
            glcm_features = extract_glcm(img)

            if des_sift is not None:
                des_sift = des_sift.flatten()[:128] 
                
                features = np.hstack((
                    des_sift,
                    color_hist.flatten(),
                    list(glcm_features.values())
                ))

                x.append(features)
                y.append(idx)

x = np.array(x)
y = np.array(y)

In [55]:
data = pd.DataFrame(x)
data['label'] = y

print(y)

data['label']

[0 0 0 ... 3 3 3]


0        0
1        0
2        0
3        0
4        0
        ..
11067    3
11068    3
11069    3
11070    3
11071    3
Name: label, Length: 11072, dtype: int32

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)

y_train_pred = rf_model.predict(x_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(classification_report(y_train, y_train_pred))

y_test_pred = rf_model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")
print(classification_report(y_test, y_test_pred))

Training Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1679
           1       1.00      1.00      1.00      1712
           2       1.00      1.00      1.00      2285
           3       1.00      1.00      1.00      2074

    accuracy                           1.00      7750
   macro avg       1.00      1.00      1.00      7750
weighted avg       1.00      1.00      1.00      7750

Test Accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       745
           1       0.99      1.00      1.00       756
           2       0.99      0.99      0.99       963
           3       1.00      1.00      1.00       858

    accuracy                           1.00      3322
   macro avg       1.00      1.00      1.00      3322
weighted avg       1.00      1.00      1.00      3322



In [57]:
#Feature Matching

# flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5),dict(checks=50))

# bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)


In [58]:
# def match_fruit(descriptors):
#     best_match = 0
#     best_match_data = None

#     for folder_path in os.listdir('Train/'):
#         for img_path in os.listdir('Train/'+folder_path):
#             img = cv2.imread('Train/'+folder_path+'/'+img_path)

#             if img is not None:
#                 final_img, img_hist = preprocess_img(img)
#                 kp_sift, des_sift= extract_features(final_img)
#                 des_sift = np.float32(des_sift)

#                 matches = flann.knnMatch(descriptors,des_sift,k=2)

#                 good_matches = []

#                 curr_match = 0
#                 for i, (fn,sm) in enumerate(matches):
#                     if fn.distance < 0.75 * sm.distance:
#                         good_matches.append(fn)

#                 hist_score = cv2.compareHist(target_hist, img_hist, cv2.HISTCMP_CORREL)
                
#                 img_glcm = extract_glcm(img)

#                 glcm_similarity = 1 - np.linalg.norm(
#                     np.array(list(target_glcm.values())) -
#                     np.array(list(img_glcm.values()))
#                 )

#                 if (len(good_matches)>4):
#                     src = np.float32([target_kp_sift[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
#                     dst = np.float32([kp_sift[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                    
#                     _, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

#                     matches_mask = mask.ravel().tolist()
#                     curr_match = sum(matches_mask)

#                     final_match = (hist_score * 0.33) + (curr_match * 0.33) + (glcm_similarity * 0.33)
#                     if best_match < final_match:
#                         best_match = final_match
#                         best_match_data={
#                             'img': img,
#                             'kp': kp_sift,
#                             'desc': des_sift,
#                             'matches': good_matches,
#                             'matches_mask':matches_mask
#                         }
    
#     result = cv2.drawMatches(
#         cv2.resize(target_img,(256,256)),
#         target_kp_sift,
#         cv2.resize(best_match_data['img'],(256,256)),
#         best_match_data['kp'],
#         best_match_data['matches'],
#         None,
#         matchesMask=best_match_data['matches_mask'],
#     )

#     cv2.imshow('result',result)
#     cv2.waitKey(0)

# # match_fruit(target_desc)